In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras import layers, models
from keras.datasets import mnist
from keras.models import Model
from IPython.display import display

In [ ]:
def preprocess(ds):
    ds = ds.astype("float32") / 255.0
    ds = np.reshape(ds, (len(ds), 28, 28, 1))
    return ds

In [ ]:
def add_noise(ds, noise_value = 0.3):
    ds_noisy = ds + noise_value * np.random.randn(*ds.shape)
    ds_noisy = np.clip(ds_noisy, 0., 1.)
    
    return ds_noisy

In [ ]:
def show_results(train_ds, noisy_ds, pred_ds=None):
    n = 5  
    indices = np.random.randint(min(len(train_ds), len(noisy_ds)), size=n)

    train_images = train_ds[indices, :]
    noisy_images = noisy_ds[indices, :]
    
    if pred_ds is not None:
        pred_images = pred_ds[indices, :]

    plt.figure(figsize=(16, 6))
    
    for i in range(n):
        ax = plt.subplot(3 if pred_ds is not None else 2, n, i + 1)
        plt.imshow(train_images[i].reshape(28, 28), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == 0:
            ax.set_ylabel("Original", size=12)

        ax = plt.subplot(3 if pred_ds is not None else 2, n, i + 1 + n)
        plt.imshow(noisy_images[i].reshape(28, 28), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == 0:
            ax.set_ylabel("Noisy", size=12)

        if pred_ds is not None:
            ax = plt.subplot(3, n, i + 1 + 2*n)
            plt.imshow(pred_images[i].reshape(28, 28), cmap='gray')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            if i == 0:
                ax.set_ylabel("Denoised", size=12)
    
    plt.tight_layout()
    plt.show()


In [ ]:
def get_model(train_ds, test_ds):
    input_shape = (28, 28, 1)

    denoise_model = models.Sequential([
        layers.Input(shape=input_shape),

        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2), padding="same"),
        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2), padding="same"),

        layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"),
        layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"),
        layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same"),
    ])

    denoise_model.compile(optimizer="adam", loss="mse")

    denoise_model.fit(
        x=train_ds,
        y=train_ds,
        epochs=5,
        batch_size=128,
        shuffle=True,
        validation_data=(test_ds, test_ds),
    )
    
    return denoise_model

In [ ]:
def run_pipeline(train_ds, test_ds, noise_value=0.3):
    noisy_train_ds = add_noise(train_ds)
    noisy_test_ds = add_noise(test_ds)
    
    print(f"Training model with noise value of {noise_value}")
    denoise_model = get_model(noisy_train_ds, noisy_test_ds)
    
    predictions = denoise_model.predict(noisy_test_ds)
    show_results(noisy_train_ds, noisy_test_ds, predictions)
    print("\n\n\n")

In [ ]:
(train_ds, _), (test_ds, _) = mnist.load_data()

train_ds = preprocess(train_ds)
test_ds = preprocess(test_ds)

for i in np.arange(0, 1.1, 0.1):
    run_pipeline(train_ds, test_ds, i)